In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [15]:
#loadig the imdb dataset

max_features = 10000 #define the vocabulary size

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) 

print(f" training data shape:{X_train.shape}, Testing data shape: {X_test.shape}")
print(f" training data shape:{y_train.shape}, Testing data shape: {y_test.shape}")

 training data shape:(25000,), Testing data shape: (25000,)
 training data shape:(25000,), Testing data shape: (25000,)


In [16]:
#sample review and its label
sample_review = X_train[0]
sample_label = y_train[0]

In [17]:
#mapping of index to their respective words tin understand the sentence

word_index = imdb.get_word_index()
reversed_word_index = {value:key for key,value in word_index.items()}

In [18]:
decoded_sample_review = " ".join([reversed_word_index.get(i-3, "?") for i in sample_review])
decoded_sample_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [19]:
max_len = 500

X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen= max_len)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [20]:
#creating the simple rnn model
dim = 128  #feature representation 
model = Sequential()
model.add(Embedding(max_features,output_dim=dim,input_length = max_len))  #Embedding layers
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer = 'adam', metrics = ["accuracy"],loss="binary_crossentropy")

In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
early_stopping  = EarlyStopping(patience=5,monitor="val_loss", restore_best_weights=True)

log_dir = "logs/fit"
tensorboard =  TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
#training the model 
history = model.fit(
    X_train,y_train, epochs=50,batch_size=32,
    validation_data=(X_test,y_test),
    callbacks =[early_stopping]
)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 148s 187ms/step - accuracy: 0.6206 - loss: 9.5049 - val_accuracy: 0.7910 - val_loss: 0.4475
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 189s 242ms/step - accuracy: 0.8145 - loss: 0.4126 - val_accuracy: 0.6970 - val_loss: 0.5639
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 161s 205ms/step - accuracy: 0.6409 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 106s 136ms/step - accuracy: 0.5020 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 109s 139ms/step - accuracy: 0.4976 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 202s 258ms/step - accuracy: 0.5005 - loss: nan - val_accuracy: 0.5000 - val_loss: nan


In [ ]:
model.save("simple_rnn.h5")
model.save("simple_rnn.keras")